In [34]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext,functions as F
from pyspark.sql.types import *

In [4]:
conf = SparkConf()
conf.setMaster("local[*]").setAppName("into_sql")

In [6]:
sc = SparkContext.getOrCreate(conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/13 09:12:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark = SQLContext(sc)

/home/saputra/belajar/data-batch-processing/belajar_batch/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [99]:
sc.stop()

In [76]:
df_green = spark.read.option("header",True).parquet("./data/pq/green/*/*")

In [75]:
df_yellow = spark.read.option("header",True).parquet("./data/pq/yellow/*/*")

In [12]:
df_green.count()

8348567

In [13]:
df_yellow.count()

122586321

In [16]:
df_yellow.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', IntegerType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True)])

In [82]:
df_yellow.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [81]:
df_green.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

# Intersect
##### - We look same column

In [24]:
set(df_yellow.columns) & set(df_green.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

# Merge Different Column 

In [79]:
df_green = df_green.withColumnRenamed('lpep_pickup_datetime','pickup_datetime').withColumnRenamed("lpep_dropoff_datetime","dropoff_datetime")

In [80]:

df_yellow = df_yellow.withColumnRenamed('tpep_pickup_datetime','pickup_datetime').withColumnRenamed("tpep_dropoff_datetime","dropoff_datetime")

In [83]:
intersect_column = []

yellow_column = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_column:
        intersect_column.append(col)

In [84]:
df_green.select(intersect_column).show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+
|       1|2019-01-23 20:18:11|2019-01-23 20:24:59|                 N|         1|         244|         116|              1|          1.2|        6.5|  0.5|    0.5|      1.55|         0.0|                  0.3|        9.35|           

# Create Flag data

In [85]:
df_green = df_green.select(intersect_column).withColumn("service_type",F.lit("green"))

In [86]:
df_yellow = df_yellow.select(intersect_column).withColumn("service_type",F.lit("yellow"))

In [87]:
df_trips_data = df_green.union(df_yellow)

In [88]:
df_trips_data.head(5)

[Row(VendorID=1, pickup_datetime=datetime.datetime(2019, 1, 23, 20, 18, 11), dropoff_datetime=datetime.datetime(2019, 1, 23, 20, 24, 59), store_and_fwd_flag='N', RatecodeID=1, PULocationID=244, DOLocationID=116, passenger_count=1, trip_distance=1.2, fare_amount=6.5, extra=0.5, mta_tax=0.5, tip_amount=1.55, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=9.35, payment_type=1, congestion_surcharge=None, service_type='green'),
 Row(VendorID=2, pickup_datetime=datetime.datetime(2019, 1, 25, 7, 58, 26), dropoff_datetime=datetime.datetime(2019, 1, 25, 9, 7, 36), store_and_fwd_flag='N', RatecodeID=1, PULocationID=55, DOLocationID=55, passenger_count=1, trip_distance=10.0, fare_amount=44.5, extra=0.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=45.3, payment_type=2, congestion_surcharge=None, service_type='green'),
 Row(VendorID=2, pickup_datetime=datetime.datetime(2019, 1, 10, 12, 9, 53), dropoff_datetime=datetime.datetime(2019, 1, 10, 12, 

In [45]:
df_trips_data.groupBy("service_type").count().show()

+------------+---------+
|service_type|    count|
+------------+---------+
|       green|  8348567|
|      yellow|122586321|
+------------+---------+



# View Table SQL

In [89]:
df_trips_data.registerTempTable("trips_data")

/home/saputra/belajar/data-batch-processing/belajar_batch/lib/python3.9/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [49]:
spark.sql("""
    SELECT service_type,count(1) 
    FROM trips_data 
    GROUP BY service_type
""").show()

+------------+---------+
|service_type| count(1)|
+------------+---------+
|       green|  8348567|
|      yellow|122586321|
+------------+---------+



In [94]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

# 1 , 2 , 3 refers to select attribute

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


# Reduce partition to save less partition

In [96]:
df_result.coalesce(1).write.parquet('./data/report/revenue',mode="overwrite")